In [37]:
import numpy as np
import re

In [38]:
def read_adj_file(file = './dummy_adjascency.txt'):
    A = []
    with open(file , 'r', encoding = 'utf-8') as f:
        for line in f:
            line2 = line.rstrip()
            line2 = line2.split('\t')
            #print(line2)
            line2 = re.sub('[(|)|\n]', '', line) # remove (,) and \n characters 
            #line2 = [i for i in line2.split(' ') if i != ''] # tokenize line
            line2 = line2.split('\t')
            #print(line2)
            if line2 != []:
                A.append(line2)
    return np.asarray(A, dtype = np.float)

fIdx = 2
rawA = read_adj_file(file = './assign-4-testData/dataset'+str(fIdx)+'.txt')
print(rawA.shape, rawA)

(9, 9) [[0. 1. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]]


In [39]:
def to_stochastic(A):
    ''' DESCRIPTION: THis function rectifies adjascency matrix A to being irreducible, stochastic and aperdiodic.
                     It adds a link from every page to all others, and make sure the matrix is stochastic.
        RETURNS: A (np 2D array): rectified adjascency matrix A.
    '''
    A_ = A.copy()
    # Sum all line elemnts to see which line is 0 aka dangling page
    normalizer = np.sum(A_, axis =1)
    # Find all non dnagling pages
    idxs = np.where(normalizer!=0)[0]
    # Find all dangling pages
    nullIdxs = np.where(normalizer==0)[0]
    normalizer[nullIdxs]= A_.shape[0]
    # Compute all the existing non null transition probs (excercise requirements, not necessary result)
    initA = A / normalizer.reshape(A_.shape[0],1)
    # Add a uniform probality to all such lines
    A_[nullIdxs,: ] = 1 
    # Reshape divider into a column vectors so each row of A is divided by each row of normalizer
    A_ /= normalizer.reshape(A_.shape[0],1)
    #print(normalizer, A_ )
    return A_, initA

def to_irreducible(A, d=0.9):
    
    return (1-d) + d* A.transpose()

A, initA = to_stochastic(rawA)
irredA = to_irreducible(A)

print("Initial matrix: ", initA)
print("\nStochastic A: ", A)
print("\nIrreducible A: ", irredA)

Initial matrix:  [[0.         0.5        0.         0.         0.         0.
  0.         0.5        0.        ]
 [0.         0.         0.33333333 0.         0.         0.
  0.         0.33333333 0.33333333]
 [0.         0.         0.         1.         0.         0.
  0.         0.         0.        ]
 [0.         0.33333333 0.         0.         0.33333333 0.
  0.         0.         0.33333333]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.5        0.
  0.         0.         0.5       ]
 [0.33333333 0.         0.         0.         0.         0.
  0.         0.33333333 0.33333333]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         1.         0.
  0.         0.         0.        ]]

Stochastic A:  [[0.         0.5        0.         0.         0.         0.
  0.         0.5        0.        ]
 [0.  

In [40]:
def power_iterate(A, d= 0.9, epsilon = 0.0001, maxIters = 2):
    # Innit page rank as 1 over the num of pages
    P_k = np.ones((A.shape[0])) / A.shape[0]
    print(P_k, P_k.shape, A.shape)
    k = 1
    delta = 1
    while (delta > epsilon) and (k <= maxIters):
        #print(k)
        P_k_1 = (1-d) + d* np.matmul(A.transpose(), P_k)
        k += 1
        delta = np.linalg.norm(P_k_1 - P_k, 1)
        P_k = P_k_1
    return P_k_1

def compute_rank(A, P, d = 0.9):
    PR = np.zeros((P.shape))
    print(P.shape,A.shape, A[:,2].shape)
    for i, p in enumerate(P):
        PR[i] = (1-d) + d * np.sum(A[:,i]* P)
    return PR

P = power_iterate(A)
PR = compute_rank(A,P)
print(*['Page: {}, Rank: {:.2f}\n'.format(i, r) for i,r in enumerate(PR)])
print('(', *[r for r in PR])

[0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
 0.11111111 0.11111111 0.11111111] (9,) (9, 9)
(9,) (9, 9) (9,)
Page: 0, Rank: 0.23
 Page: 1, Rank: 0.36
 Page: 2, Rank: 0.27
 Page: 3, Rank: 0.38
 Page: 4, Rank: 0.68
 Page: 5, Rank: 0.18
 Page: 6, Rank: 0.18
 Page: 7, Rank: 0.40
 Page: 8, Rank: 0.48

( 0.2307222222222222 0.35872222222222216 0.2717222222222222 0.37888888888888883 0.6792222222222221 0.18438888888888888 0.18438888888888888 0.40405555555555545 0.47588888888888875


In [41]:
def write_to_file(initA, A, irredA, PR, path = './Assignment4_Results/result.txt'):
    args = [initA, A, irredA, PR]
    with open(path, 'w', encoding ='utf-8') as f:
        
        for idx, arg in enumerate(args):
            m = arg
            s = '\n(' if idx != 0 else '('
            f.write(s)
            for i in range(m.shape[0]):
                if len(m.shape)> 1:
                    print(*['{:.2f}'.format(a) for a in m[i,:] ])
                    f.write(' '.join((['{:.2f}'.format(a) for a in m[i,:] ])))
                    s = '\n' if i != m.shape[0]-1 else ')'
                else:
                    f.write(' '.join((['{:.2f}'.format(m[i])])))
                    s = ' ' if i != m.shape[0]-1 else ')'
                f.write(s)
        
# StudentID1_StudentID2_FirstName1_Lastname1_FirstName2_Lastname2.zip     
# myId = 91
write_to_file(initA, A, irredA, PR, path = './Assignment4_Results/result'+str(fIdx)+'.txt')

0.00 0.50 0.00 0.00 0.00 0.00 0.00 0.50 0.00
0.00 0.00 0.33 0.00 0.00 0.00 0.00 0.33 0.33
0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00
0.00 0.33 0.00 0.00 0.33 0.00 0.00 0.00 0.33
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 0.50 0.00 0.00 0.00 0.50
0.33 0.00 0.00 0.00 0.00 0.00 0.00 0.33 0.33
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00
0.00 0.50 0.00 0.00 0.00 0.00 0.00 0.50 0.00
0.00 0.00 0.33 0.00 0.00 0.00 0.00 0.33 0.33
0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00
0.00 0.33 0.00 0.00 0.33 0.00 0.00 0.00 0.33
0.11 0.11 0.11 0.11 0.11 0.11 0.11 0.11 0.11
0.00 0.00 0.00 0.00 0.50 0.00 0.00 0.00 0.50
0.33 0.00 0.00 0.00 0.00 0.00 0.00 0.33 0.33
0.11 0.11 0.11 0.11 0.11 0.11 0.11 0.11 0.11
0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00
0.10 0.10 0.10 0.10 0.20 0.10 0.40 0.20 0.10
0.55 0.10 0.10 0.40 0.20 0.10 0.10 0.20 0.10
0.10 0.40 0.10 0.10 0.20 0.10 0.10 0.20 0.10
0.10 0.10 1.00 0.10 0.20 0.10 0.10 0.20 0.10
0.10 0.10 